## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing Pieces

### B.1 Import a Single Piece and Check Metadata for Title and Composer

- Here you will want to select the appropriate 'prefix' that identifies the location of your file.
- `'Music_Files/'` is for files in the local notebook; `'https://crimproject.org/mei/'` is for the files on CRIM.
- Then provide the full name (and extension) of your music file, such as `'CRIM_Model_0038.mei'`

In [141]:
# Select a prefix:
# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/' 


# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# These join the strings and import the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

{'title': 'Ave Maria', 'composer': 'Josquin Des Prés', 'date': 1502}


In [142]:

# piece.numberParts(piece.notes())

In [143]:
_dict = piece._getPartNumberDict()
# _dict


In [144]:
# get the P-types, ngrams as series, and the notes with detail index
# must pass the same "n" to mel and ptypes
ngram_length = 4

p_types = piece.presentationTypes(melodic_ngram_length=ngram_length)
# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

# voice numbers

_dict = piece._getPartNumberDict()


In [145]:
# the function
def ptype_ema_table(row, ngrams_series, notes_det, _dict):
    offsets = row["Offsets"]
    voices = row["Voices"]
    pairs = list(zip(offsets, voices))

    ng_list = []
    for pair in pairs:
        offset = pair[0]
        voice = pair[1]
        ngram_coordinates = ngrams_series[(ngrams_series.index.get_level_values(0) == offset) & (ngrams_series.index.get_level_values(2) == voice)]
        ngram_coordinates = ngram_coordinates.reset_index()
        ngram_coordinates = ngram_coordinates.rename(columns={'level_2' : 'Voice'})
        ng_list.append(ngram_coordinates)
    ng_df = pd.concat(ng_list)
    ng_df_first = ng_df.set_index('First')
    ng_df_last = ng_df.set_index('Last')

    merged_df = ng_df_first.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
    merged_df.reset_index(inplace=True)
    merged_df2 = ng_df_last.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
    merged_df2.reset_index(inplace=True)
    cols_to_use = merged_df2.columns.difference(merged_df.columns)
    final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
    final['PartNumber'] = final['Voice'].map(_dict)

    

    return final
    
             

In [148]:
p_types['result'] = p_types.apply(lambda row: ptype_ema_table(row, ngrams_series, notes_det, _dict), axis=1)
# p_types
# # sample result
# # for item in p_types['result']:
# #     print(item)
# # p_types

test = p_types.iloc[0]['result'].set_index(['First', 'Last'])
test


,,Voice,0,FirstMeasure,FirstBeat,LastBeat,LastMeasure,PartNumber
First,Last,,,,,,,
0.0,20.0,[Superius],"(P4, P1, M2, M2)",1,1.0,3.0,3,1
16.0,36.0,Altus,"(P4, P1, M2, M2)",3,1.0,3.0,5,2
32.0,52.0,Tenor,"(P4, P1, M2, M2)",5,1.0,3.0,7,3
48.0,68.0,Bassus,"(P4, P1, M2, M2)",7,1.0,3.0,9,4


In [44]:
# the function
def hr_ema_table(row, ngrams_series, notes_det, _dict):
    offset = row["Offset"]
    voices = row["Voices"]
    pairs = []
    for voice in voices:
        pair = (offset, voice)
        pairs.append(pair)

    ng_list = []
    for pair in pairs:
        offset = pair[0]
        voice = pair[1]
        ngram_coordinates = ngrams_series[(ngrams_series.index.get_level_values(0) == offset) & (ngrams_series.index.get_level_values(2) == voice)]
        ngram_coordinates = ngram_coordinates.reset_index()
        ngram_coordinates = ngram_coordinates.rename(columns={'level_2' : 'Voice'})
        ng_list.append(ngram_coordinates)
    ng_df = pd.concat(ng_list)
    ng_df_first = ng_df.set_index('First')
    ng_df_last = ng_df.set_index('Last')

    merged_df = ng_df_first.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
    merged_df.reset_index(inplace=True)
    merged_df2 = ng_df_last.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
    merged_df2.reset_index(inplace=True)
    cols_to_use = merged_df2.columns.difference(merged_df.columns)
    final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
    final.reset_index(inplace=True)
    final.rename(columns={0 : "ngram"}, inplace=True)
    final['PartNumber'] = final['Voice'].map(_dict)
    final = final[["First", "Last", "PartNumber", "FirstMeasure", "FirstBeat", "LastMeasure", "LastBeat"]]
    

    return final

In [45]:
# get the following first

# must pass the same "n" to mel and hr
ngram_length = 4

# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

# voice numbers

_dict = piece._getPartNumberDict()

# get the hr and clean cols
hr = piece.homorhythm(ngram_length = ngram_length).reset_index()
hr = hr.rename(columns={"Measure" : "FirstMeasure", "Beat": "FirstBeat", "hr_voices" : "Voices"})



# Cadences

In [116]:
# for cadences

# get the following first

# must pass the same "n" to mel.  It's the default for cvfs anyway
ngram_length = 3

# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

# voice numbers

_dict = piece._getPartNumberDict()

In [177]:
# LAST offsets of ngrams
ngrams_last = ngrams.reset_index(level=0)
ngrams_last.index.rename('Offset', inplace=True)
ngrams_last = ngrams_last.drop(['First'], axis=1)

# FIRST offsets of ngrams

ngrams_first = ngrams.reset_index(level=1)
ngrams_first.index.rename('Offset', inplace=True)
ngrams_first = ngrams_first.drop(['Last'], axis=1)

#now merging
merged_df = ngrams_last.merge(notes_det, left_index=True, right_index=True, how='left')
merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
merged_df.reset_index(inplace=True)
merged_df.rename(columns={"Offset" : "Last"}, inplace=True)
merged_df2 = ngrams_first.merge(notes_det, left_index=True, right_index=True, how='left')
merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
merged_df2.reset_index(inplace=True)
merged_df2.rename(columns={"Offset" : "First"}, inplace=True)
cols_to_use = merged_df2.columns.difference(merged_df.columns)
final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
final = final.set_index(["First", "Last"])
final = final[['FirstMeasure', 'FirstBeat', 'LastBeat', 'LastMeasure']]
final.head(3)


    

,,FirstMeasure,FirstBeat,LastBeat,LastMeasure
First,Last,,,,
0.0,20.0,3,3.0,1.0,1
4.0,24.0,4,1.0,3.0,1
16.0,36.0,5,3.0,1.0,3


In [103]:
final_2 = cvfs.merge(final, left_index=True, right_index=True, how='left')
final_2.rename(columns=_dict, inplace=True)
final_2.fillna('-')
final_2 = final_2.reset_index()
# final_3 = final_3.columns({"Offset" : "Last"})

final_2.rename(columns={'Offset': "Last"}, inplace=True)
final_2 = final_2.set_index(['First', 'Last'])
final_2

1    2    3    4  FirstMeasure  FirstBeat  LastBeat  \
First  Last                                                            
188.0  188.0   NaN    T    C    b          24.0        3.0       3.0   
NaN    232.0     C    u    T    b           NaN        NaN       NaN   
       272.0     C    T  NaN  NaN           NaN        NaN       NaN   
       304.0   NaN    A    C    T           NaN        NaN       NaN   
416.0  416.0     C  NaN    T    B          53.0        1.0       4.0   
472.0  472.0     C    T    B  NaN          60.0        1.0       3.0   
512.0  512.0     t  NaN    C    T          65.0        1.0       3.0   
NaN    608.0     C  NaN    T    B           NaN        NaN       NaN   
640.0  640.0     T    C  NaN  NaN          81.0        1.0       1.0   
664.0  664.0   NaN  NaN    T    C          84.0        1.0       4.0   
       664.0   NaN  NaN    T    C          84.0        1.0       1.0   
688.0  688.0     T    C  NaN  NaN          87.0        1.0       1.0   
736.0  736.0   NaN  NaN    C    T          93.0        1.0       3.0   
928.0  928.0     C    L    T    B         109.0        1.0       2.5   
1008.0 1008.0  NaN  NaN    T    C         119.0        1.0       1.0   
NaN    1072.0  NaN  NaN    T    C           NaN        NaN       NaN   
1096.0 1096.0    A    z    b    T         130.0        1.0       1.0   
1098.0 1096.0    A    z    b    T         130.0        1.0       1.0   
1148.0 1148.0    T    C  NaN  NaN         136.0        3.0       1.0   
NaN    1184.0  NaN    A  NaN    T           NaN        NaN       NaN   

               LastMeasure  
First  Last                 
188.0  188.0          25.0  
NaN    232.0           NaN  
       272.0           NaN  
       304.0           NaN  
416.0  416.0          53.0  
472.0  472.0          61.0  
512.0  512.0          66.0  
NaN    608.0           NaN  
640.0  640.0          82.0  
664.0  664.0          84.0  
       664.0          85.0  
688.0  688.0          88.0  
736.0  736.0          94.0  
928.0  928.0         110.0  
1008.0 1008.0        121.0  
NaN    1072.0          NaN  
1096.0 1096.0        131.0  
1098.0 1096.0        131.0  
1148.0 1148.0        138.0  
NaN    1184.0          NaN

In [99]:
piece.emaAddresses(final_2)

First   Last  
178.0   188.0     23-24/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
181.0   188.0     23-24/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
216.0   232.0     28-30/1+2+3+4+FirstMeasure+FirstBeat+LastBeat+...
224.0   232.0     29-30/1+2+3+4+FirstMeasure+FirstBeat+LastBeat+...
225.0   232.0     29-30/1+2+3+4+FirstMeasure+FirstBeat+LastBeat+...
262.0   272.0     33-35/1+2+FirstMeasure+FirstBeat+LastBeat+Last...
265.0   272.0     34-35/1+2+FirstMeasure+FirstBeat+LastBeat+Last...
294.0   304.0     37-39/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
297.0   304.0     38-39/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
408.0   416.0     52-53/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
409.0   416.0     52-53/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
464.0   472.0     59-60/1+2+3+FirstMeasure+FirstBeat+LastBeat+La...
465.0   472.0     59-60/1+2+3+FirstMeasure+FirstBeat+LastBeat+La...
504.0   512.0     64-65/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
505.0   512.0     64-65/1+3+4+Fir

In [98]:
piece.presentationTypes()


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps,result
0,Josquin Des Prés,Ave Maria,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[P-8, P1, P-8]","[0.0, 16.0, 32.0, 48.0]","[(4, 1, 2, 2)]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,0.0,None,0,First Last Voice 0 Firs...
1,Josquin Des Prés,Ave Maria,56.0,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[P-8, P1, P-8]","[56.0, 72.0, 88.0, 104.0]","[(-2, -2, -2, 2)]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,0.0,None,0,First Last Voice 0 ...
2,Josquin Des Prés,Ave Maria,124.0,"[16/3.0, 18/3.0, 20/3.0, 22/3.0]","[P-8, P1, P-8]","[124.0, 140.0, 156.0, 172.0]","[(1, 1, 2, 2)]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",PEN,4,False,0.0,None,0,First Last Voice 0 Fir...
3,Josquin Des Prés,Ave Maria,176.0,"[23/1.0, 28/1.0]",[P-8],"[176.0, 216.0]","[(-3, -2, 2, 2)]",[40.0],"[Altus, Bassus]",FUGA,2,False,0.0,None,0,First Last Voice 0 First...
4,Josquin Des Prés,Ave Maria,244.0,"[31/3.0, 35/3.0]",[P-8],"[244.0, 276.0]","[(1, 2, 2, 1)]",[32.0],"[[Superius], Tenor]",FUGA,2,False,0.0,None,0,First Last Voice 0 Fir...
5,Josquin Des Prés,Ave Maria,428.0,"[54/3.0, 55/1.0, 59/3.0, 60/1.0]","[P-5, P-4, P-5]","[428.0, 432.0, 468.0, 472.0]","[(4, -2, 2, 2)]","[4.0, 36.0, 4.0]","[[Superius], Altus, Tenor, Bassus]",ID,4,False,0.0,None,0,First Last Voice 0 Fi...
6,Josquin Des Prés,Ave Maria,508.0,"[64/3.0, 66/3.0, 67/3.0, 69/3.0]","[P-5, P-4, P-5]","[508.0, 524.0, 532.0, 548.0]","[(2, -3, 2, -3)]","[16.0, 8.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",ID,4,False,0.0,None,0,First Last Voice 0 Fi...
7,Josquin Des Prés,Ave Maria,616.0,"[78/1.0, 81/1.0]",[P-8],"[616.0, 640.0]","[(1, 1, 2, -3)]",[24.0],"[[Superius], Tenor]",FUGA,2,False,0.0,None,0,First Last Voice 0 Fir...
8,Josquin Des Prés,Ave Maria,748.0,"[94/1.0, 98/1.0]",[P1],"[748.0, 796.0]","[(1, 1, 2, -2)]",[48.0],"[Altus, Altus]",FUGA,2,False,0.0,None,1,First Last Voice 0 FirstMea...
9,Josquin Des Prés,Ave Maria,944.0,"[111/1.0, 114/1.0, 119/1.0, 122/1.0]","[P-8, P8, P-8]","[944.0, 968.0, 1008.0, 1032.0]","[(1, 1, 2, -2)]","[24.0, 40.0, 24.0]","[[Superius], Tenor, [Superius], Tenor]",ID,4,False,0.0,None,0,First Last Voice 0 F...
